<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/06_pytorch_transfer_learning_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06. Bài tập PyTorch Transfer Learning

Chào mừng bạn đến với notebook template bài tập 06. PyTorch Transfer Learning.

Có một số câu hỏi trong notebook này và mục tiêu của bạn là trả lời chúng bằng cách viết code Python và PyTorch.

> **Lưu ý:** Có thể có nhiều hơn một giải pháp cho mỗi bài tập, đừng quá lo lắng về câu trả lời *chính xác* tuyệt đối. Hãy cố gắng viết một số code hoạt động trước rồi cải thiện nó nếu có thể.

## Tài nguyên và giải pháp

* Những bài tập/giải pháp này dựa trên [phần 06. PyTorch Transfer Learning](https://www.learnpytorch.io/06_pytorch_transfer_learning/) của khóa học Learn PyTorch for Deep Learning bởi Zero to Mastery.

**Giải pháp:** 

Hãy cố gắng hoàn thành code bên dưới *trước khi* xem những cái này.

* Xem [walkthrough trực tiếp của các giải pháp (kể cả lỗi) trên YouTube](https://youtu.be/ueLolShyFqs).
* Xem ví dụ [notebook giải pháp cho những bài tập này trên GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/06_pytorch_transfer_learning_exercise_solutions.ipynb).

## 1. Make predictions trên toàn bộ test dataset và vẽ confusion matrix cho kết quả của mô hình chúng ta so với truth labels.
* **Lưu ý:** Bạn sẽ cần lấy dataset và trained model/retrain model từ notebook 06 để thực hiện predictions.
* Xem [03. PyTorch Computer Vision section 10](https://www.learnpytorch.io/03_pytorch_computer_vision/#10-making-a-confusion-matrix-for-further-prediction-evaluation) để có ý tưởng.

In [ ]:
# Import các thư viện/code cần thiết
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms, datasets

# Thử lấy torchinfo, cài đặt nếu không hoạt động
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Thử import thư mục going_modular, tải xuống từ GitHub nếu không hoạt động
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Lấy going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 1708, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 1708 (delta 67), reused 151 (delta 60), pack-reused 1548
Receiving objects: 100% (1708/1708), 230.85 MiB | 14.36 MiB/s, done.
Resolving deltas: 100% (927/927), done.
Checking out files: 100% (124/124), done.


In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Lấy dữ liệu

In [ ]:
import os
import requests
import zipfile

from pathlib import Path

# Thiết lập đường dẫn đến thư mục dữ liệu
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Nếu thư mục hình ảnh không tồn tại, tải xuống và chuẩn bị nó...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    # Tải xuống dữ liệu pizza, steak, sushi
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Giải nén dữ liệu pizza, steak, sushi
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...") 
        zip_ref.extractall(image_path)

    # Xóa file .zip
    os.remove(data_path / "pizza_steak_sushi.zip")

# Thiết lập thư mục
train_dir = image_path / "train"
test_dir = image_path / "test"

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


### Chuẩn bị dữ liệu

In [ ]:
# Tạo transforms pipeline
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape tất cả hình ảnh thành 224x224 (mặc dù một số mô hình có thể yêu cầu kích thước khác)
    transforms.ToTensor(), # 2. Chuyển giá trị hình ảnh thành giữa 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. Mean là [0.485, 0.456, 0.406] (trên mỗi colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. Standard deviation là [0.229, 0.224, 0.225] (trên mỗi colour channel),
])

In [ ]:
# Tạo training và testing DataLoader cũng như lấy danh sách class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=simple_transform, # resize, chuyển hình ảnh thành giữa 0 & 1 và normalize chúng
                                                                               batch_size=32) # đặt mini-batch size thành 32

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f5f520c9bd0>,
 ['pizza', 'steak', 'sushi'])

### Lấy và chuẩn bị pretrained model

In [ ]:
# Thiết lập mô hình với pretrained weights và gửi đến target device
model_0 = torchvision.models.efficientnet_b0(pretrained=True).to(device)
#model_0 # uncomment để output (nó rất dài)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [ ]:
# Freeze tất cả base layers trong phần "features" của mô hình (feature extractor) bằng cách đặt requires_grad=False
for param in model_0.features.parameters():
    param.requires_grad = False

In [ ]:
# Đặt manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Lấy độ dài của class_names (một output unit cho mỗi class)
output_shape = len(class_names)

# Tạo lại classifier layer và seed nó đến target device
model_0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=output_shape, # cùng số lượng output units như số lượng classes của chúng ta
                    bias=True)).to(device)

### Train mô hình

In [ ]:
# Định nghĩa loss và optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr=0.001)

In [ ]:
# Đặt random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Bắt đầu timer
from timeit import default_timer as timer 
start_time = timer()

# Thiết lập training và lưu kết quả
model_0_results = engine.train(model=model_0,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=5,
                       device=device)

# Kết thúc timer và in ra thời gian mất
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0894 | train_acc: 0.4492 | test_loss: 0.9214 | test_acc: 0.5085
Epoch: 2 | train_loss: 0.8697 | train_acc: 0.7734 | test_loss: 0.8036 | test_acc: 0.7434
Epoch: 3 | train_loss: 0.7769 | train_acc: 0.7734 | test_loss: 0.7404 | test_acc: 0.7737
Epoch: 4 | train_loss: 0.7244 | train_acc: 0.7422 | test_loss: 0.6488 | test_acc: 0.8864
Epoch: 5 | train_loss: 0.6426 | train_acc: 0.7812 | test_loss: 0.6254 | test_acc: 0.8968
[INFO] Total training time: 31.032 seconds


### Make predictions trên toàn bộ test dataset với mô hình

In [ ]:
# TODO

### Tạo confusion matrix với test preds và truth labels

Cần các thư viện sau để tạo confusion matrix:
* torchmetrics - https://torchmetrics.readthedocs.io/en/stable/
* mlxtend - http://rasbt.github.io/mlxtend/

In [ ]:
# Xem torchmetrics có tồn tại không, nếu không, cài đặt nó
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Lưu ý: Nếu bạn đang sử dụng Google Colab, điều này có thể yêu cầu restart runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")

     |████████████████████████████████| 409 kB 7.5 MB/s 
     |████████████████████████████████| 1.3 MB 45.7 MB/s 
mlxtend version: 0.19.0


In [ ]:
# Import mlxtend phiên bản upgraded
import mlxtend 
print(mlxtend.__version__)
assert int(mlxtend.__version__.split(".")[1]) >= 19 # nên là phiên bản 0.19.0 hoặc cao hơn

0.19.0


In [ ]:
# TODO

## 2. Lấy "most wrong" của predictions trên test dataset và vẽ 5 hình ảnh "most wrong". Bạn có thể làm điều này bằng cách:
* Predicting trên tất cả test dataset, lưu trữ labels và predicted probabilities.
* Sắp xếp predictions theo *wrong prediction* và sau đó *descending predicted probabilities*, điều này sẽ cho bạn wrong predictions với *highest* prediction probabilities, nói cách khác, "most wrong".
* Vẽ top 5 hình ảnh "most wrong", tại sao bạn nghĩ mô hình đã nhầm những cái này?

Bạn sẽ muốn:
* Tạo DataFrame với sample, label, prediction, pred prob
* Sắp xếp DataFrame theo correct (label == prediction không)
* Sắp xếp DataFrame theo pred prob (descending)
* Vẽ top 5 "most wrong" image predictions

In [ ]:
# TODO

## 3. Predict trên hình ảnh pizza/steak/sushi của riêng bạn - mô hình hoạt động như thế nào? Điều gì xảy ra nếu bạn predict trên một hình ảnh không phải pizza/steak/sushi?
* Ở đây bạn có thể lấy hình ảnh từ một website như http://www.unsplash.com để thử hoặc bạn có thể upload hình ảnh của riêng mình.

In [ ]:
# TODO: Lấy hình ảnh pizza/steak/sushi


In [ ]:
# TODO: Lấy hình ảnh không phải pizza/steak/sushi

## 4. Train mô hình từ phần 4 trong notebook 06 part 3 lâu hơn (10 epochs nên đủ), điều gì xảy ra với hiệu suất?

* Xem mô hình trong notebook 06 part 3 để tham khảo: https://www.learnpytorch.io/06_pytorch_transfer_learning/#3-getting-a-pretrained-model

In [ ]:
# TODO: Tạo lại mô hình mới

In [ ]:
# TODO: Train mô hình trong 10 epochs

## 5. Train mô hình từ phần 4 ở trên với nhiều dữ liệu hơn, nói 20% hình ảnh từ Food101 của Pizza, Steak và Sushi images.
* Bạn có thể tìm thấy [20% Pizza, Steak, Sushi dataset](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/data/pizza_steak_sushi_20_percent.zip) trên GitHub của khóa học. Nó được tạo với notebook [`extras/04_custom_data_creation.ipynb`](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/04_custom_data_creation.ipynb).

### Lấy dữ liệu 20%

In [ ]:
import os
import requests
import zipfile

from pathlib import Path

# Thiết lập đường dẫn đến thư mục dữ liệu
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi_20_percent"
image_data_zip_path = "pizza_steak_sushi_20_percent.zip"

# Nếu thư mục hình ảnh không tồn tại, tải xuống và chuẩn bị nó...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    # Tải xuống dữ liệu pizza, steak, sushi
    with open(data_path / image_data_zip_path, "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Giải nén dữ liệu pizza, steak, sushi 20%
    with zipfile.ZipFile(data_path / image_data_zip_path, "r") as zip_ref:
        print("Unzipping pizza, steak, sushi 20% data...") 
        zip_ref.extractall(image_path)

    # Xóa file .zip
    os.remove(data_path / image_data_zip_path)

# Thiết lập thư mục
train_dir_20_percent = image_path / "train"
test_dir_20_percent = image_path / "test"

train_dir_20_percent, test_dir_20_percent

Did not find data/pizza_steak_sushi_20_percent directory, creating one...
Unzipping pizza, steak, sushi 20% data...


(PosixPath('data/pizza_steak_sushi_20_percent/train'),
 PosixPath('data/pizza_steak_sushi_20_percent/test'))

### Tạo DataLoaders

In [ ]:
# Tạo transforms pipeline
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape tất cả hình ảnh thành 224x224 (mặc dù một số mô hình có thể yêu cầu kích thước khác)
    transforms.ToTensor(), # 2. Chuyển giá trị hình ảnh thành giữa 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. Mean là [0.485, 0.456, 0.406] (trên mỗi colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. Standard deviation là [0.229, 0.224, 0.225] (trên mỗi colour channel),
])

In [ ]:
# Tạo training và testing DataLoader cũng như lấy danh sách class names
train_dataloader_20_percent, test_dataloader_20_percent, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                                     test_dir=test_dir_20_percent,
                                                                                                     transform=simple_transform, # resize, chuyển hình ảnh thành giữa 0 & 1 và normalize chúng
                                                                                                     batch_size=32) # đặt mini-batch size thành 32

train_dataloader_20_percent, test_dataloader_20_percent, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f5ede28e390>,
 ['pizza', 'steak', 'sushi'])

### Lấy pretrained model

In [ ]:
# TODO

### Train mô hình với 20% dữ liệu

In [ ]:
# TODO

## 6. Thử một mô hình khác từ [`torchvision.models`](https://pytorch.org/vision/stable/models.html) trên dữ liệu Pizza, Steak, Sushi, mô hình này hoạt động như thế nào?
* Bạn sẽ phải thay đổi kích thước của classifier layer để phù hợp với vấn đề của chúng ta.
* Bạn có thể muốn thử một EfficientNet với số cao hơn B0 của chúng ta, có thể `torchvision.models.efficientnet_b2()`?
  * **Lưu ý:** Tùy thuộc vào mô hình bạn sử dụng, bạn sẽ phải chuẩn bị/transform dữ liệu theo một cách nhất định.

In [ ]:
# TODO